In [14]:
import numpy as np
import aplpy
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams, style
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import astropy.io.fits as pyfits
from astropy import units as u
import pyregion
from astropy.wcs import WCS
from sys import argv
import time
import os
import copy
import warnings

/usr/local/lib/python2.7/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: The mpl_toolkits.axes_grid module was deprecated in version 2.1. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist provies the same functionality instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [3]:
# parameter

In [4]:
#---------------------------------------
# set the path for the forder
path_image='../Image_Data/'
path_BT = '../Image_BT/'
path_samping = '../Table_Data/'
path_figure = '../Plot_Figure/'
path_region = '../Plot_Region/'

In [67]:
#---------------------------------------
# set the outname
outname_b4       = 'n1333iras4a_B4_uv24_2900kl_sm200_100mas_noshift'
outname_b7       = 'n1333iras4a_B7_uv24_2900kl_sm200_100mas_noshift'
outname_mask     = 'n1333iras4a_B7_uv24_2900kl_sm200_100mas_noshift_mask_I'
outname_mask_4A1 = 'n1333iras4a_B7_uv24_2900kl_sm200_100mas_noshift_mask_4A1_I'
outname_mask_4A2 = 'n1333iras4a_B7_uv24_2900kl_sm200_100mas_noshift_mask_4A2_I'

#---------------------------------------
# set the output filename
def slt_imageName(outname):
    I_imageName          = '%s%s_I.image.fits'%(path_image, outname)
    Q_imageName          = '%s%s_Q.image.fits'%(path_image, outname)
    U_imageName          = '%s%s_U.image.fits'%(path_image, outname)
    PI_imageName         = '%s%s_PI.image.fits'%(path_image, outname)
    PI_cut3PI_imageName  = '%s%s_PI_cut3PI.image.fits'%(path_image, outname)
    PA_imageName         = '%s%s_PA.image.fits'%(path_image, outname)
    Per_imageName        = '%s%s_Per.image.fits'%(path_image, outname)
    Per_cut3PI_imageName = '%s%s_Per_cut3PI.image.fits'%(path_image, outname)
    BT_imageName         = '%s%s_BT.fits'%(path_BT, outname) 
    
    return I_imageName, Q_imageName, U_imageName, PI_imageName, PI_cut3PI_imageName,\
            PA_imageName, Per_imageName, Per_cut3PI_imageName, BT_imageName
    

BT_B4_imageName      = '%s%s_BT.fits'%(path_BT, outname_b4)
BT_B7_imageName      = '%s%s_BT.fits'%(path_BT, outname_b7)
BT_slt_imageName     = '%s%s_BT.fits'%(path_BT, outname_b4)
BT_div_imageName     = '%sBT_B4_div_BT_B7.fits'%(path_BT)
Mask_imageName       = '%s%s.image.fits'%(path_image, outname_mask)
Mask_4A1_imageName   = '%s%s.image.fits'%(path_image, outname_mask_4A1)
Mask_4A2_imageName   = '%s%s.image.fits'%(path_image, outname_mask_4A2)

#----
BT_B4            = '%s%s_BT.fits'%(path_BT, outname_b4)
BT_B7            = '%s%s_BT.fits'%(path_BT, outname_b7)
BT_slt_image     = '%s%s_BT.fits'%(path_BT, outname_b4)
BT_div_image     = '%sBT_B4_div_BT_B7.fits'%(path_BT)
Mask_image       = '%s%s.image.fits'%(path_image, outname_mask)
Mask_4A1_image   = '%s%s.image.fits'%(path_image, outname_mask_4A1)
Mask_4A2_image   = '%s%s.image.fits'%(path_image, outname_mask_4A2)

In [63]:
#---------------------------------------
# set the 
dist = 293 # [pc]
sampling_all = 10

ra_4A1 = 52.2939125
dec_4A1 = 31.2252611
ra_4A2 = 52.2934625
dec_4A2 = 31.2255806
ra_4A1_4A2 = 52.2937125
dec_4A1_4A2 = 31.2255431

In [12]:
#---------------------------------------
# set the rms and criteria
# ALMA Band 4
rms_I_b4       = 2.3e-4              #(Jy beam-1)
rms_Q_b4       = 1.2e-5              #(Jy beam-1)
rms_PI_b4      = 1.2e-5              #(Jy beam-1)
I_clip_b4      = 3*rms_I_b4          #(Jy beam-1)
PI_lowclip_b4  = 3*rms_PI_b4         #(Jy beam-1)
PI_highclip_b4 = 1e99*rms_PI_b4      #(Jy beam-1)
sampling_b4    = sampling_all
PI_vmax_b4     = 2.849e-3            #(Jy beam-1)
convert_unit_b4 = 'Jy2mJy'

# ALMA Band 7
rms_I_b7       = 6.4e-4              #(Jy beam-1)
rms_Q_b7       = 4.2e-5              #(Jy beam-1)
rms_PI_b7      = 4.2e-5              #(Jy beam-1)
I_clip_b7      = 3*rms_I_b7          #(Jy beam-1)
PI_lowclip_b7  = 3*rms_PI_b7         #(Jy beam-1)
PI_highclip_b7 = 1e99*rms_PI_b7      #(Jy beam-1)
sampling_b7    = sampling_all
PI_vmax_b7     = 1.005e-2            #(Jy beam-1)
convert_unit_b7 = 'Jy2mJy'

# function for select parameter
def slt_parm(outname):
    
    if outname == outname_b4:
        rms_I=rms_I_b4
        rms_Q=rms_Q_b4
        rms_PI=rms_PI_b4
        I_clip=I_clip_b4
        PI_lowclip=PI_lowclip_b4
        PI_highclip=PI_highclip_b4
        sampling=sampling_b4
        PI_vmax=PI_vmax_b4
        convert_unit=convert_unit_b4
    elif outname == outname_b7:
        rms_I=rms_I_b7
        rms_Q=rms_Q_b7
        rms_PI=rms_PI_b7
        I_clip=I_clip_b7
        PI_lowclip=PI_lowclip_b7
        PI_highclip=PI_highclip_b7
        sampling=sampling_b7
        PI_vmax=PI_vmax_b7
        convert_unit=convert_unit_b7
        
    return rms_I, rms_Q, rms_PI, I_clip, PI_lowclip, PI_highclip, \
            sampling, PI_vmax, convert_unit

In [ ]:
# ALMA Band 4
scale_10percent_b4  = 0.1e-2
scale_line_b4       = 0.006
seg_color_b4        = 'blue'
seg_width_b4        = 2
seg_color_base_b4   = 'white'
seg_width_base_b4   = 3
    
# ALMA Band 7
scale_10percent_b7  = 0.1e-2
scale_line_b7       = 0.008
seg_color_b7        = 'black'
seg_width_b7        = 4
seg_color_base_b7   = 'white'
seg_width_base_b7   = 5

# function for select parameter
def slt_seg(outname):
    if outname == outname_b4:
        scale_10percent=scale_10percent_b4
        scale_line=scale_line_b4
        seg_color=seg_color_b4
        seg_width=seg_width_b4
        seg_color_base=seg_color_base_b4
        seg_width_base=seg_width_base_b4
    elif outname == outname_b7:
        scale_10percent=scale_10percent_b7
        scale_line=scale_line_b7
        seg_color=seg_color_b7
        seg_width=seg_width_b7
        seg_color_base=seg_color_base_b7
        seg_width_base=seg_width_base_b7
        
    return scale_10percent, scale_line, seg_color, seg_width, \
            seg_color_base, seg_width_base

In [62]:
# get the header
outname = outname_b4
I_image = '%s%s_I.image.fits'%(path_image, outname)
I_data = pyfits.getdata(I_image)
I_hd = pyfits.getheader(I_image)

# get the parameter from the header
hd_freq = I_hd['RESTFRQ']           # frequency [Hz]
hd_bmaj = I_hd['BMAJ']              # [deg]
hd_bmin = I_hd['BMIN']              # [deg]
hd_bpa = I_hd['BPA']                # [deg]

hd_cell = np.abs(I_hd['CDELT1'])    # cell size [deg]
hd_ra = I_hd['CRVAL1']              # ra [deg]
hd_dec = I_hd['CRVAL2']             # dec [deg]
hd_date = I_hd['DATE']              # date [y-m-dTh:m:s]
hd_pixelNum = I_hd['NAXIS1']        # pixel number per axis [pixel]

# calculate the sampling number
sampling = np.divide(hd_bmin, hd_cell*2)
sampling_int = np.around(sampling)

# print the results
#print sampling_int

In [61]:
'''
print hd_freq
print hd_bmaj, hd_bmin, hd_bpa
print hd_bmin*3600
print hd_cell*3600
sampling = np.divide(hd_bmin, hd_cell*2)
sampling_int = np.around(sampling)
print sampling
print sampling_int
print hd_pixelNum
print hd_ra, hd_dec
print hd_date
'''

'\nprint hd_freq\nprint hd_bmaj, hd_bmin, hd_bpa\nprint hd_bmin*3600\nprint hd_cell*3600\nsampling = np.divide(hd_bmin, hd_cell*2)\nsampling_int = np.around(sampling)\nprint sampling\nprint sampling_int\nprint hd_pixelNum\nprint hd_ra, hd_dec\nprint hd_date\n'